## Tutorial for `datamanager` Using `ExtArgs`

This tutorial demonstrates use of datamanager using the `ExtArgs` pattern, which is implicitly used when calling datamanager from Snakemake with the `script` syntax or when using the CLI. This is arguably the preferred approach in most cases since it ensures consistency: any problems present using this interface would also appear when interfacing with datamanager from Snakemake or CLI.

*NOTE*: If using VSCode then Pylance uses a different Python interpreter than the one executing the notebook. Even if it is using the correct conda environment (it should be!), the changes to `syspath` here aren't reflected in the Python interpreter running Pylance. So, to avoid Pylance complaining about datamanager not being found and enable auto-completion, one has to add the path explicitly into settings. See, this [StackExchange post](https://stackoverflow.com/questions/65252074/import-path-to-own-script-could-not-be-resolved-pylance-reportmissingimports). Also, remember to remove at some indeterminate point in the future.

### Contents of this tutorial

* [Setup and Overview](#setup-and-overview)
* [Run Management](#run-management)



### Setup and Overview

There is an `ExtArgs` Pydantic model defined in `cmd_interface/external.py`. You should instantiate one of these objects for each "command". Then that object should be passed as the last (third) argument to `entry_point()`.

Due to datamanager not being in a public package directory, we will have to ensure `syspath` allows us to import as expected (it's assumed this notebook is being run from some subpath under the EXIOBASE installation). Along with other standard stuff, this is done below.

In [2]:
import os
import sys
import re
import time
import warnings

from pathlib import Path
from pprint import pprint
from datetime import datetime

# Turn on all warnings... https://docs.python.org/3/library/warnings.html

if not sys.warnoptions:
    warnings.simplefilter("default")  # Change the filter in this process
    os.environ["PYTHONWARNINGS"] = "default"  # Also affect subprocesses

cur_working_dir = os.getcwd()
print(f"Current working directory: {cur_working_dir}")

# Iterate back until we find the EXIOBASE install notice, or we hit the max
# number of allowed iterations.
cur_check_dir = Path(cur_working_dir).resolve()
b_found_install = False
for i in range(0, 15):
    cur_check_dir = cur_check_dir.parent
    if cur_check_dir.is_dir() and Path(cur_check_dir / "exiobase_install_notice.txt").is_file():
        # Found root dir for EXIOBASE install
        b_found_install = True
        print(f"Found EXIOBASE install directory at: {cur_check_dir}")
        break

if not b_found_install:
    raise Exception("Failed to find EXIOBASE installation!")

datamanager_dir = str(Path(cur_check_dir / "repo" / "datamanager").resolve())
if not datamanager_dir in sys.path:
    sys.path.append(datamanager_dir)
pprint(sys.path)

# The required imports
import datamanager
from datamanager import entry_point
from datamanager.cmd_interface.external import ExtArgs



Current working directory: /home/peterma/exio_dev/repo/docs
Found EXIOBASE install directory at: /home/peterma/exio_dev
['/home/peterma/exio_dev/repo/docs',
 '/home/peterma/miniconda3/envs/exio-datamanager/lib/python310.zip',
 '/home/peterma/miniconda3/envs/exio-datamanager/lib/python3.10',
 '/home/peterma/miniconda3/envs/exio-datamanager/lib/python3.10/lib-dynload',
 '',
 '/home/peterma/miniconda3/envs/exio-datamanager/lib/python3.10/site-packages',
 '/home/peterma/exio_dev/repo/datamanager']
'In datamanager/__init__.py, conda env: exio-datamanager'


In [ ]:
    # input_files: List[Path] = []
    # output_files: List[Path] = []
    # simple_command: constr(
    #     strict=True,
    #     to_lower=True,
    #     min_length=1,
    #     max_length=int(g_persistent.constraints.simple_command_str_max_len),
    #     pattern=g_persistent.constraints.simple_command_str_regex,
    # )  # type: ignore
    # error_log_file: Path | None
    # warning_log_file: Path | None
    # info_log_file: Path | None
    # debug_log_file: Path | None
    # runid: constr(
    #     strict=True,
    #     to_lower=True,
    #     min_length=1,
    #     max_length=int(g_persistent.constraints.runid_str_max_len),
    #     pattern=g_persistent.constraints.runid_str_regex,
    # ) | None  # type: ignore
    # run_type: RuntypeEnum | None
    # run_owner: str | None
    # exio_version: str | None
    # query_cmd: constr(
    #     strict=True,
    #     to_lower=True,
    #     min_length=1,
    #     max_length=int(g_persistent.constraints.query_str_str_max_len),
    #     pattern=g_persistent.constraints.query_str_str_regex,
    # ) | None  # type: ignore
    # query_str: constr(min_length=1) | None  # type: ignore
    # query_output_format: QueryOutputFormat | None  # type: ignore

    # # What follows are argument specific to certain sections of code
    # http_dl_allow_overwrite: bool | None
    # http_dl_fail_exist_but_no_meta: FileButNoMetaActionEnum = FileButNoMetaActionEnum(
    #     "fail"
    # )

### Run Management

query_runids_args = ExtArgs()